# Import packages

In this file, we'll need `numpy`, `matplotlib`, `pandas` and a handy tool from `sklearn`. We'll also be using a gpx parser for python, `gpxpy`, which can be installed by entering the following into a cell:
```python
!pip install gpxpy
```

In [ ]:
#%pylab inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# change above to "notebook" for 3d viewing excellence

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import minmax_scale
import pandas as pd
import gpxpy

In [ ]:
from datetime import datetime

# Load Strava data

Find the file on your hard drive and store its location to the variable `gpx_fn`. Open the file using Python's `open` and then parse this file with the gpx parser. 

In [ ]:
gpx_fn = './Sortie_v_lo_matinale.gpx'
gpx_file = open(gpx_fn, 'r')
sortie = gpxpy.parse(gpx_file)

Each strava file contains a single track with a single segment that is broken up into many points. Therefore, the course (`course`) we have ridden is just the set of points of our trip (`sortie`).

In [ ]:
def getMinMax(x):
    return (x.min(), x.max())

In [ ]:
def getRange(v):
    return v[1] - v[0]

In [ ]:
mu_lat = np.mean(getMinMax(course['lat']))
mu_lon = np.mean(getMinMax(course['lon']))

In [ ]:
halfRange = np.maximum(getRange(getMinMax(course['lat'])), getRange(getMinMax(course['lon'])))/2

## Make elevation plot as a function of lat-lon

Lastly, use the `Axes3D` object to plot a 3D elevation plot of the route. 

In [ ]:
fig = plt.figure(figsize=(10,6))
ax = plt.gca(projection='3d')
for j in range(latlonsegs.shape[0]):
    if j in [0, latlonsegs.shape[0]-1]:
        ax.plot(latlonsegs[j, :, 0], latlonsegs[j, :, 1], latlonsegs[j, :, 2], c=myCmap[j,:], 
                label='t={}'.format(datetime.fromtimestamp(course['ts'].values[j]).time()))
    else:
        ax.plot(latlonsegs[j, :, 0], latlonsegs[j, :, 1], latlonsegs[j, :, 2], c=myCmap[j,:])
ax.view_init(25, 225)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_zlabel('Elevation')
ax.set_zlim(0, 120);
ax.legend();
plt.axis('tight');
ax.set_title('Morning ride to UBC');

## Notes

It should be possible to do this using these tools below

In [ ]:
# from matplotlib.colors import ListedColormap, BoundaryNorm
# from mpl_toolkits.mplot3d.art3d import Line3DCollection

but for the life of me I cannot figure out how. 

# Another attempt at plotting

In [ ]:
# duration in seconds
print('Duration = {} min'.format(sortie.get_duration()/60))
sortie.get_elevation_extremes()

In [ ]:
course = sortie.tracks[0].segments[0].points

Format the information of the points as a data frame so that we have easier access to the information. 

In [ ]:
from mpl_toolkits.mplot3d.art3d import Line3DCollection

In [ ]:
from matplotlib.colors import ListedColormap, BoundaryNorm

To do: view from azimuth = -90 (S), azim = -180 (W) and elev=90

In [ ]:
course = pd.DataFrame([[pt.latitude, pt.longitude, pt.elevation, pt.time.timestamp()] 
                       for pt in course], columns=('lat', 'lon', 'elev', 'ts'))

In [ ]:
course.head()

# Plot data

## Colourmap stuff

For plotting reasons, I've wanted to define pair of functions that stores the colour information in columns in the data frame. Therefore, I will use `cmapFromTimestamp` to compute a colourmap from the timestamp information and use `addNewColumnsInPlace` to insert the colour information into the `course` dataframe. 

In [ ]:
def addNewColumnsInPlace(colNames, data, func, **kwargs):
    res = func(data, **kwargs)
    for j, cn in enumerate(colNames):
        data[cn] = res[:, j]
    return

In [ ]:
def cmapFromTimestamp(data, cmap='Blues', columnName='ts', scale_range=(0,1)):
    return plt.cm.cmap_d[cmap](minmax_scale(data[columnName], 
                                            feature_range=scale_range))

In [ ]:
colourCols = ['cmap_' + x for x in 'rgba']
addNewColumnsInPlace(colourCols, course, cmapFromTimestamp, 
                     cmap='GnBu', scale_range=(.5,1))

Above, we have created four new columns, `{{print('{}, {}, {}, {}'.format(*colourCols))}}`, that contain the colour map information for our data frame.

## More colourmap stuff

Instead of using the colourmaps developed above, I want to use a line plot, so I have to do this nonsense in order to colour each indvidual little segment.